# 07 菜品分类-数据分组/数据透视表

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
# 一个cell输出多行语句
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 一、数据分组
根据一个或多个键（函数、数组或df列名）将数据分成**若干组**，然后对分组后的数据分别进行**汇总计算**，并将汇总后的结果进行**合并**。

被用作汇总计算的函数称为聚合函数。

数据分组流程如下：

<img src='./image/groupby_flow.jpg' width='60%'>

Excel实现：前提要先对分组的那一列（键）进行排序（升序或降序）

<img src='./image/groupby_excel.jpg' width='50%'>

选中分组区域，单击**数据**-》**分类汇总**

<img src='./image/groupby_excel2.jpg' width='60%'>

Excel常见的分类汇总方式：

<img src='./image/groupby_excel3.jpg' width='60%'>

Python实现：**groupby()**方法

### 1.1 分组键是列名

#### 1.1.1 按照一列进行分组

In [4]:
df = pd.read_csv('./data/train-pivot.csv', encoding='gbk')
df

,用户ID,客户分类,区域,是否省会,7月销量,8月销量,9月销量
0,59224,A类,一线城市,是,6,20,0
1,55295,B类,三线城市,否,37,27,35
2,46035,A类,二线城市,是,8,1,8
3,2459,C类,一线城市,是,7,8,14
4,22179,B类,三线城市,否,9,12,4
5,22557,A类,二线城市,是,42,20,55


In [7]:
df.groupby('客户分类')
# 返回的不是DataFrame对象，而是DataFrameGroupBy对象
# 里面包含着分组以后的若干组数据，但是没有直接显示出来，需要进行汇总计算后才会显示

# 对分组进行计数运算
# 如果对分组进行数值运算，只有数据类型是数值（int，float）的列才会参与运算
df.groupby('客户分类').count()

# 求和运算
df.groupby('客户分类').sum()

# 这些函数称为聚合函数，《开始烹调-数据运算》中汇总运算都适用

,用户ID,区域,是否省会,7月销量,8月销量,9月销量
客户分类,,,,,,
A类,3,3,3,3,3,3
B类,2,2,2,2,2,2
C类,1,1,1,1,1,1


,用户ID,7月销量,8月销量,9月销量
客户分类,,,,
A类,127816,56,41,63
B类,77474,46,39,39
C类,2459,7,8,14


#### 1.1.2 按照多列进行分组

In [10]:
df.groupby(['客户分类', '区域']).count()
df.groupby(['客户分类', '区域']).sum()

# 有时候不需要对所有的列进行汇总运算，方法：将想要计算的列通过索引方式取出来，再在该列上进行汇总
df.groupby('客户分类')['用户ID'].count()

用户ID  是否省会  7月销量  8月销量  9月销量
客户分类 区域                                
A类   一线城市     1     1     1     1     1
     二线城市     2     2     2     2     2
B类   三线城市     2     2     2     2     2
C类   一线城市     1     1     1     1     1

用户ID  7月销量  8月销量  9月销量
客户分类 区域                           
A类   一线城市  59224     6    20     0
     二线城市  68592    50    21    63
B类   三线城市  77474    46    39    39
C类   一线城市   2459     7     8    14

客户分类
A类    3
B类    2
C类    1
Name: 用户ID, dtype: int64

### 1.2 分组键是Series
DataFrame取出一列就是Series，比如df['客户分类']就是一个Series

#### 1.2.1 按照一个Series进行分组

In [11]:
df.groupby(df['客户分类']).count()

,用户ID,区域,是否省会,7月销量,8月销量,9月销量
客户分类,,,,,,
A类,3,3,3,3,3,3
B类,2,2,2,2,2,2
C类,1,1,1,1,1,1


In [12]:
df.groupby([df['客户分类'], df['区域']]).sum()

用户ID  7月销量  8月销量  9月销量
客户分类 区域                           
A类   一线城市  59224     6    20     0
     二线城市  68592    50    21    63
B类   三线城市  77474    46    39    39
C类   一线城市   2459     7     8    14

In [13]:
df.groupby(df['客户分类'])['用户ID'].count()

客户分类
A类    3
B类    2
C类    1
Name: 用户ID, dtype: int64

### 1.3 神奇的aggregate方法
前面用到的聚合函数都是在DataFrameGroupBy上面调用的，分组后所有的列都做同一种运算，一次只能使用一种汇总方式

aggregate一次可以使用多种方式

In [14]:
# 先做计数，再做求和运算
df.groupby('客户分类').aggregate(['count', 'sum'])
# 对用户ID进行计数，对7、8月销量进行求和
df.groupby('客户分类').aggregate({'用户ID': 'count', '7月销量':'sum', '8月销量': 'sum'})

用户ID            区域                是否省会       7月销量      8月销量      9月销量  \
     count     sum count           sum count  sum count sum count sum count   
客户分类                                                                          
A类       3  127816     3  一线城市二线城市二线城市     3  是是是     3  56     3  41     3   
B类       2   77474     2      三线城市三线城市     2   否否     2  46     2  39     2   
C类       1    2459     1          一线城市     1    是     1   7     1   8     1   

          
     sum  
客户分类      
A类    63  
B类    39  
C类    14

,用户ID,7月销量,8月销量
客户分类,,,
A类,3,56,41
B类,2,46,39
C类,1,7,8


### 1.4 对分组后的结果重置索引
DataFrameGroupBy对象经过汇总运算以后的形式并不是标准的DataFrame形式

通过**reset_index()**方法将非标准转化为标准DataFrame形式

In [15]:
df.groupby('客户分类').sum()
df.groupby('客户分类').sum().reset_index()

,用户ID,7月销量,8月销量,9月销量
客户分类,,,,
A类,127816,56,41,63
B类,77474,46,39,39
C类,2459,7,8,14


,客户分类,用户ID,7月销量,8月销量,9月销量
0,A类,127816,56,41,63
1,B类,77474,46,39,39
2,C类,2459,7,8,14


## 二、数据透视表
数据透视表和数据分组类似但又不同，数据分组在**一维（行）方向**上不断拆分，而数据透视表是在**行、列方向**上同时拆分

Excel实现：

<img src='./image/pivot_excel.jpg' />

Python实现：

pd.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')

* data：表示要做数据透视表的整个表
* values：对应Excel中值那个框
* index：对应Excel中行那个框
* columns：对应Excel列那个框
* aggfunc：表示对values的计算类型
* fill_value：表示对空值的填充值
* margins：表示是否显示合计列
* dropna：表示是否删除缺失，如果为真时，则把一整行全作为缺失值删除
* margins_name：表示合计列的列名

In [21]:
# 客户分类作为index，区域作为columns，用户ID作为values，对values执行count运算
pd.pivot_table(df, values='用户ID', columns='区域', index='客户分类', 
               aggfunc='count', margins=True, margins_name='总计',
               fill_value=0)

区域,一线城市,三线城市,二线城市,总计
客户分类,,,,
A类,1,0,2,3
B类,0,2,0,2
C类,1,0,0,1
总计,2,2,2,6


In [25]:
# 对不同值进行不同的计算
pd.pivot_table(df, values=['用户ID', '7月销量'], columns='区域',
               index='客户分类', aggfunc={'用户ID':'count', '7月销量': 'sum'})
pd.pivot_table(df, values=['用户ID', '7月销量'], columns='区域',
               index='客户分类', aggfunc={'用户ID':'count', '7月销量': 'sum'}, fill_value=0).reset_index()

7月销量             用户ID          
区域   一线城市  三线城市  二线城市 一线城市 三线城市 二线城市
客户分类                                
A类    6.0   NaN  50.0  1.0  NaN  2.0
B类    NaN  46.0   NaN  NaN  2.0  NaN
C类    7.0   NaN   NaN  1.0  NaN  NaN

客户分类 7月销量           用户ID          
区域      一线城市 三线城市 二线城市 一线城市 三线城市 二线城市
0    A类    6    0   50    1    0    2
1    B类    0   46    0    0    2    0
2    C类    7    0    0    1    0    0